In [2]:
import torch 

import torchvision as tv 

import time 

import matplotlib.pyplot as plt

In [3]:
#loading dataset
BATCH_SIZE = 256 

train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)

train = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE)

/Users/aleksandrsvistunov/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
for batch in train:
    print(type(batch[0]))
    break

<class 'torch.Tensor'>


In [42]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 2048),
    torch.nn.ReLU(), 
    torch.nn.BatchNorm1d(2048),
    torch.nn.Linear(2048, 1024),
    torch.nn.ReLU(), 
    torch.nn.BatchNorm1d(1024),
    torch.nn.Linear(1024, 512),
    torch.nn.ReLU(), 
    torch.nn.BatchNorm1d(512),
    torch.nn.Linear(512, 10)
)

In [43]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr=0.006)
num_epochs = 20

In [230]:
def train_model():
    for ep in range(num_epochs + 1):
        train_iters, train_passed = 0, 0
        train_loss, train_acc = 0., 0. 
        start = time.time()
        
        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(axis=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed = 0, 0
        test_loss, test_acc = 0., 0. 
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(axis=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
        
        print('ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}'.format(
        ep, time.time() - start, train_loss / train_iters, train_acc / train_passed, 
        test_loss / test_iters, test_acc / test_passed)
             )
            

In [44]:
train_model()

ep: 0, taked: 27.509, train_loss: 0.49582242306242597, train_acc: 0.8196666666666667, test_loss: 0.47002851627767084, test_acc: 0.8253
ep: 1, taked: 26.712, train_loss: 0.3735350763544123, train_acc: 0.86125, test_loss: 0.46261726804077624, test_acc: 0.8284
ep: 2, taked: 27.134, train_loss: 0.3417379951223414, train_acc: 0.8731166666666667, test_loss: 0.42383682560175656, test_acc: 0.8518
ep: 3, taked: 27.088, train_loss: 0.3190451115369797, train_acc: 0.88165, test_loss: 0.42140161748975513, test_acc: 0.8552
ep: 4, taked: 27.929, train_loss: 0.3018202029010083, train_acc: 0.8883, test_loss: 0.44944524988532064, test_acc: 0.8484
ep: 5, taked: 27.686, train_loss: 0.28500760159593946, train_acc: 0.8940833333333333, test_loss: 0.7891166875138879, test_acc: 0.8631
ep: 6, taked: 27.471, train_loss: 0.2725510811552088, train_acc: 0.8987166666666667, test_loss: 0.5847410719841719, test_acc: 0.8615
ep: 7, taked: 28.607, train_loss: 0.26146716316963764, train_acc: 0.9033166666666667, test_loss:

### Convolutional NN

In [4]:
import torch 

In [114]:
X = torch.randn([1, 1, 8, 8])

In [51]:
X[0][0]

tensor([[-0.2906,  0.4880, -0.6926, -2.3897,  0.3771, -0.6239,  0.1224, -0.8395],
        [-1.3610, -0.1927, -0.3939,  1.6888,  1.4555,  0.0314, -0.1710, -0.7583],
        [ 1.1036, -0.5579,  0.0708,  2.0836,  1.0304,  0.8713, -1.3628, -0.5483],
        [ 1.7149, -0.3713, -0.9609,  0.2689, -0.2862, -1.7155,  0.4723, -1.2409],
        [-1.2308, -0.1191, -0.4264,  0.0873,  0.7643, -2.0886,  0.3779, -1.0321],
        [-1.7581, -0.7506,  0.1579, -0.2088,  0.2930, -0.0335, -1.1956, -0.1251],
        [-0.4938,  0.2531, -0.4120, -0.4505, -0.5430, -0.5885, -1.1072, -0.4508],
        [ 0.6124, -0.7455, -0.2885, -0.2311,  1.3411, -0.6279,  0.7520,  1.0228]])

In [89]:
def show_prety(matrix):
    torch.set_printoptions(profile="short")
    print(matrix)
    torch.set_printoptions(profile="default")
    return None 

show_prety(torch.nn.functional.pad(X[0][0], pad=[1, 1,]))

tensor([[ 0.,  0.,  1.,  2.,  3.,  0.],
        [ 0.,  4.,  5.,  6.,  7.,  0.],
        [ 0.,  8.,  9., 10., 11.,  0.],
        [ 0., 12., 13., 14., 15.,  0.]])


In [67]:
conv2d = torch.nn.Conv2d(1, 1, kernel_size=3, padding=1)
conv2d(X).shape

torch.Size([1, 1, 8, 8])

In [35]:
conv2d = torch.nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
conv2d(X).shape

torch.Size([1, 1, 4, 4])

In [70]:
conv2d.weight.shape

torch.Size([1, 1, 3, 3])

In [77]:
conv2d = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
X = torch.randn([1, 3, 8, 8])
conv2d(X).shape

torch.Size([1, 32, 8, 8])

In [78]:
conv2d.weight.shape

torch.Size([32, 3, 3, 3])

### Poolling 

In [80]:
X = torch.arange(16).reshape([1, 1, 4, 4]).float()

show_prety(torch.nn.functional.pad(X[0][0], pad=[1, 1, 1, 1]))

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  2.,  3.,  0.],
        [ 0.,  4.,  5.,  6.,  7.,  0.],
        [ 0.,  8.,  9., 10., 11.,  0.],
        [ 0., 12., 13., 14., 15.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]])


In [85]:
pool2d = torch.nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [106]:
pool2d = torch.nn.AvgPool2d(3, padding=(1, 1), stride=1)
pool2d(X)

tensor([[[[ 1.1111,  2.0000,  2.6667,  2.0000],
          [ 3.0000,  5.0000,  6.0000,  4.3333],
          [ 5.6667,  9.0000, 10.0000,  7.0000],
          [ 4.6667,  7.3333,  8.0000,  5.5556]]]])

## Model 

In [144]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(1, 6, kernel_size=5, padding=2),
    torch.nn.ReLU(), 
    torch.nn.MaxPool2d(2, stride=2),
    torch.nn.Conv2d(6, 12, kernel_size=5),
    torch.nn.ReLU(), 
    torch.nn.MaxPool2d(2, stride=2),
    torch.nn.Flatten(),
    torch.nn.Linear(300, 50), 
    torch.nn.ReLU(),
    torch.nn.Linear(50, 10)
)

In [145]:
model

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=300, out_features=50, bias=True)
  (8): ReLU()
  (9): Linear(in_features=50, out_features=10, bias=True)
)

In [154]:
for X, y in train:
    print(X[0].shape, y[0].shape)
    break

torch.Size([1, 28, 28]) torch.Size([])


In [147]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr=0.006)
num_epochs = 20

In [148]:
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed = 0, 0
        train_loss, train_acc = 0., 0. 
        start = time.time()
        
        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(axis=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed = 0, 0
        test_loss, test_acc = 0., 0. 

        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(axis=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
        
        print('ep: {}, taken: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}'.format(
        ep, time.time() - start, train_loss / train_iters, train_acc / train_passed, 
        test_loss / test_iters, test_acc / test_passed)
             )
            

In [149]:
train_model()

ep: 0, taked: 14.260, train_loss: 0.654350513093015, train_acc: 0.7583166666666666, test_loss: 0.44713825099170207, test_acc: 0.8387
ep: 1, taked: 14.265, train_loss: 0.39536155297401104, train_acc: 0.8581833333333333, test_loss: 0.38414958864450455, test_acc: 0.8615
ep: 2, taked: 14.011, train_loss: 0.34246235470822517, train_acc: 0.8756833333333334, test_loss: 0.36428055390715597, test_acc: 0.8681
ep: 3, taked: 14.736, train_loss: 0.313039660326978, train_acc: 0.8864, test_loss: 0.33981270529329777, test_acc: 0.8773
ep: 4, taked: 14.087, train_loss: 0.2902521965985603, train_acc: 0.8930833333333333, test_loss: 0.34157638289034364, test_acc: 0.8763
ep: 5, taked: 13.895, train_loss: 0.2758229638033725, train_acc: 0.8992833333333333, test_loss: 0.3408873688429594, test_acc: 0.8778
ep: 6, taked: 13.973, train_loss: 0.26465216456575597, train_acc: 0.9030666666666667, test_loss: 0.3417618960142136, test_acc: 0.8769
ep: 7, taked: 14.174, train_loss: 0.25281009902345375, train_acc: 0.9069, t